In [1]:
import torch

import torch.nn as nn
torch.manual_seed(20)
ls = list()
ls.append(nn.Linear(784, 50))
ls.append(nn.Linear(50, 30))
ls.append(nn.Linear(30, 40))
ls.append(nn.Linear(40, 10))

ls_W = list()
for i in ls:
    ls_W.append(i.weight.detach().T)
    ls_W.append(i.bias)
# .tofile('./weight/blocks.bin')

In [2]:
for i in ls_W:
    print(i.shape)

torch.Size([784, 50])
torch.Size([50])
torch.Size([50, 30])
torch.Size([30])
torch.Size([30, 40])
torch.Size([40])
torch.Size([40, 10])
torch.Size([10])


In [3]:
ls_W[6][:8,:8]

tensor([[ 0.1381, -0.0392, -0.1167,  0.0576, -0.0649,  0.0762, -0.0374, -0.0546],
        [-0.0223, -0.0873,  0.0544,  0.0754, -0.1553,  0.0185, -0.1150,  0.1129],
        [-0.0857,  0.1376, -0.0815, -0.0897,  0.0014, -0.0192,  0.1514, -0.1348],
        [-0.1487,  0.0774, -0.1488, -0.0509,  0.1436, -0.1402, -0.1425,  0.0908],
        [ 0.1373,  0.0775,  0.1055,  0.0791, -0.0604, -0.1312,  0.0034,  0.0533],
        [-0.1258,  0.1252,  0.1527, -0.0910, -0.0345, -0.1377,  0.0695,  0.0361],
        [-0.0373, -0.0577, -0.1283, -0.1017,  0.0515,  0.0751, -0.0216,  0.0044],
        [-0.1199, -0.0121, -0.1258, -0.1540, -0.1112, -0.1345, -0.1343,  0.0014]])

In [4]:
for i in ls_W:
    print(i.shape)

torch.Size([784, 50])
torch.Size([50])
torch.Size([50, 30])
torch.Size([30])
torch.Size([30, 40])
torch.Size([40])
torch.Size([40, 10])
torch.Size([10])


In [5]:
for i in range(len(ls_W)):
    ls_W[i].detach().numpy().tofile('./weight/'+str(i)+'_init_blocks'+'.bin')
    

In [6]:
import torchvision

dd = torchvision.datasets.MNIST('./data',train=True)

In [7]:
mnist_data = nn.LayerNorm((784),elementwise_affine=False)(dd.data.detach().reshape(-1, 28*28).to(float))
mnist_data = mnist_data.numpy().astype('float32')

In [8]:
mnist_data.shape

(60000, 784)

In [9]:
i =128
print(mnist_data[:4,i:i+8])


[[-0.44079015 -0.44079015 -0.44079015 -0.44079015 -0.44079015 -0.44079015
  -0.44079015 -0.44079015]
 [ 1.4226017   2.5431545   1.4226017   0.1232371  -0.4728017  -0.4728017
  -0.4728017  -0.4728017 ]
 [-0.37816164 -0.37816164 -0.37816164 -0.37816164 -0.37816164 -0.37816164
  -0.37816164 -0.37816164]
 [-0.33075416 -0.33075416 -0.33075416 -0.33075416 -0.33075416 -0.33075416
  -0.33075416 -0.33075416]]


In [10]:
# mnist_data.tofile('./data/data_norm.bin')

In [11]:
# mnist_label = dd.targets.detach().numpy().astype('float32')

In [12]:
# mnist_label.tofile('./data/label.bin')

In [13]:
mnist_data[:4].shape

(4, 784)

In [14]:
n_ls  = list()
for i in ls[:-1]:
    n_ls.append(i)
    n_ls.append(nn.ReLU())

n_ls.append(ls[3])

In [15]:
model = nn.Sequential(*n_ls)
model

Sequential(
  (0): Linear(in_features=784, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=30, bias=True)
  (3): ReLU()
  (4): Linear(in_features=30, out_features=40, bias=True)
  (5): ReLU()
  (6): Linear(in_features=40, out_features=10, bias=True)
)

In [169]:
A = list()
Z = list()
Z.append(n_ls[0](torch.Tensor(mnist_data[:16])))
A.append(n_ls[1](Z[0]))
Z.append(n_ls[2](A[0]))
A.append(n_ls[3](Z[1]))
Z.append(n_ls[4](A[1]))
A.append(n_ls[5](Z[2]))
Z.append(n_ls[6](A[2]))
A.append(Z[3])



In [170]:
mnist_label = dd.targets
O = nn.Softmax(dim=1)(A[3])
print(nn.CrossEntropyLoss()(O, mnist_label[:16]))
O[8:,:8]

tensor(2.2984, grad_fn=<NllLossBackward0>)


tensor([[0.0996, 0.1178, 0.0848, 0.1139, 0.1134, 0.0906, 0.0825, 0.1123],
        [0.1026, 0.1183, 0.0866, 0.1059, 0.1101, 0.0924, 0.0840, 0.1094],
        [0.0992, 0.1183, 0.0839, 0.1074, 0.1113, 0.0942, 0.0831, 0.1115],
        [0.0982, 0.1225, 0.0827, 0.1064, 0.1092, 0.0942, 0.0855, 0.1123],
        [0.1010, 0.1161, 0.0826, 0.1120, 0.1152, 0.0918, 0.0840, 0.1107],
        [0.0999, 0.1187, 0.0838, 0.1092, 0.1141, 0.0921, 0.0840, 0.1117],
        [0.1005, 0.1166, 0.0858, 0.1103, 0.1142, 0.0897, 0.0860, 0.1099],
        [0.1032, 0.1180, 0.0847, 0.1064, 0.1109, 0.0923, 0.0838, 0.1114]],
       grad_fn=<SliceBackward0>)

In [171]:

A_0 = n_ls[0](torch.Tensor(mnist_data[:8]))
A_0 = n_ls[1](A_0)
A_0[:,:8]

tensor([[0.2013, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0569, 0.0000],
        [0.1143, 0.0000, 0.4939, 0.0000, 0.0000, 0.3471, 0.0944, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.6248, 0.0000, 0.0000, 0.0000],
        [0.1605, 0.6952, 0.0000, 0.0340, 0.5496, 0.0000, 0.0144, 0.1711],
        [0.2259, 0.1223, 0.0000, 0.0000, 0.0000, 0.0088, 0.1536, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.1839, 0.0000, 0.0000, 0.0000],
        [0.5416, 0.0383, 0.0278, 0.0000, 0.0000, 0.3027, 0.0000, 0.0000],
        [0.0062, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4925, 0.0000]],
       grad_fn=<SliceBackward0>)

In [172]:
A[2].T[8:16,:8]

tensor([[0.0953, 0.1488, 0.0276, 0.1052, 0.1098, 0.0852, 0.1387, 0.1294],
        [0.1247, 0.2620, 0.1786, 0.1021, 0.2590, 0.2786, 0.1823, 0.1476],
        [0.0766, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1419, 0.1329, 0.1130, 0.0596, 0.0484, 0.0798, 0.1312, 0.1729],
        [0.0661, 0.0912, 0.0395, 0.0000, 0.0095, 0.0150, 0.0297, 0.0627],
        [0.4032, 0.4439, 0.2506, 0.3235, 0.2949, 0.3188, 0.2001, 0.3688],
        [0.0039, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0347, 0.0930, 0.0000, 0.0000, 0.0000, 0.0000, 0.0778, 0.0000]],
       grad_fn=<SliceBackward0>)

In [173]:
dCE[:,:8]

tensor([[ 0.0988,  0.1182,  0.0870,  0.1112,  0.1146, -0.9105,  0.0815,  0.1116],
        [-0.8975,  0.1154,  0.0875,  0.1068,  0.1155,  0.0884,  0.0799,  0.1111],
        [ 0.1007,  0.1203,  0.0821,  0.1075, -0.8902,  0.0957,  0.0845,  0.1102],
        [ 0.1020, -0.8835,  0.0879,  0.1098,  0.1102,  0.0901,  0.0855,  0.1086],
        [ 0.1026,  0.1175,  0.0857,  0.1102,  0.1142,  0.0913,  0.0812,  0.1105],
        [ 0.1029,  0.1177, -0.9156,  0.1082,  0.1126,  0.0932,  0.0818,  0.1111],
        [ 0.1007, -0.8849,  0.0865,  0.1102,  0.1134,  0.0922,  0.0832,  0.1107],
        [ 0.1003,  0.1211,  0.0831, -0.8951,  0.1084,  0.0955,  0.0846,  0.1112],
        [ 0.0996, -0.8822,  0.0848,  0.1139,  0.1134,  0.0906,  0.0825,  0.1123],
        [ 0.1026,  0.1183,  0.0866,  0.1059, -0.8899,  0.0924,  0.0840,  0.1094],
        [ 0.0992,  0.1183,  0.0839, -0.8926,  0.1113,  0.0942,  0.0831,  0.1115],
        [ 0.0982,  0.1225,  0.0827,  0.1064,  0.1092, -0.9058,  0.0855,  0.1123],
        [ 0.1010

In [174]:
Y = torch.eye(10)[mnist_label[:16]]

dCE = O-Y

torch.round(torch.matmul(A[2].T, dCE)[8:16,:8]*1000) /1000
torch.sum(A[3], axis=0)


tensor([-0.3267,  2.1735, -3.0872,  0.8687,  1.3822, -1.7991, -3.3708,  1.1807,
        -1.3833, -1.4389], grad_fn=<SumBackward1>)

In [175]:
O[8:,:8]

tensor([[0.0996, 0.1178, 0.0848, 0.1139, 0.1134, 0.0906, 0.0825, 0.1123],
        [0.1026, 0.1183, 0.0866, 0.1059, 0.1101, 0.0924, 0.0840, 0.1094],
        [0.0992, 0.1183, 0.0839, 0.1074, 0.1113, 0.0942, 0.0831, 0.1115],
        [0.0982, 0.1225, 0.0827, 0.1064, 0.1092, 0.0942, 0.0855, 0.1123],
        [0.1010, 0.1161, 0.0826, 0.1120, 0.1152, 0.0918, 0.0840, 0.1107],
        [0.0999, 0.1187, 0.0838, 0.1092, 0.1141, 0.0921, 0.0840, 0.1117],
        [0.1005, 0.1166, 0.0858, 0.1103, 0.1142, 0.0897, 0.0860, 0.1099],
        [0.1032, 0.1180, 0.0847, 0.1064, 0.1109, 0.0923, 0.0838, 0.1114]],
       grad_fn=<SliceBackward0>)

In [208]:
torch.matmul(A[2].T,dCE)[8:16,:8]

tensor([[ 1.4855e-02, -2.8435e-01,  5.2957e-02, -2.4561e-01,  1.0235e-01,
         -5.3971e-02,  6.5279e-02,  1.5166e-01],
        [ 1.7984e-02, -2.8859e-01, -4.3565e-02, -1.9924e-01, -1.2070e-02,
          8.4585e-02,  4.2112e-02,  1.3610e-01],
        [ 9.2253e-03, -5.5034e-03,  8.0826e-03,  1.0337e-02,  1.0661e-02,
         -6.8279e-02,  7.6636e-03,  1.0362e-02],
        [ 3.8976e-02, -9.8948e-02,  6.4955e-02, -2.9380e-01, -2.2951e-02,
         -1.0003e-01,  4.9247e-02,  8.8140e-02],
        [-3.7952e-02,  1.3469e-03,  2.9989e-02, -6.6452e-02, -5.1667e-02,
         -1.7575e-02,  4.3578e-02,  8.2393e-03],
        [ 6.7266e-02, -3.2462e-01,  1.1157e-01, -4.1161e-01, -6.3146e-02,
         -2.1183e-01,  5.3375e-02,  1.1777e-01],
        [ 3.8623e-04,  4.6194e-04,  3.4023e-04,  4.3471e-04,  4.4796e-04,
         -3.5592e-03,  3.1873e-04,  4.3632e-04],
        [-4.9335e-02, -1.2435e-01,  3.7023e-02, -8.3024e-02,  4.9478e-02,
          4.5453e-03,  3.5935e-02,  4.8013e-02]], grad_fn=<SliceB

In [209]:
(torch.round(torch.matmul(dCE, n_ls[3*2].weight)*1000)/1000)[8:,:8]
# (n_ls[3*2].weight.T)[8:16,:8]
#  A[3][8:,:8]

tensor([[ 0.0280,  0.0830, -0.1580, -0.0900, -0.0400, -0.1320,  0.0400, -0.0840],
        [ 0.0540,  0.1510, -0.0200, -0.1580,  0.0980,  0.0290, -0.0680,  0.0150],
        [-0.0690, -0.0800,  0.0710,  0.0380, -0.0430,  0.0850,  0.0850,  0.0580],
        [-0.0880, -0.0230,  0.0010,  0.1270,  0.1680,  0.1320, -0.0920,  0.0390],
        [-0.0680, -0.0810,  0.0700,  0.0380, -0.0420,  0.0830,  0.0850,  0.0580],
        [ 0.0260,  0.1100, -0.1700,  0.1300,  0.0330, -0.0750,  0.0050,  0.0390],
        [ 0.0280,  0.0820, -0.1560, -0.0910, -0.0400, -0.1310,  0.0410, -0.0840],
        [ 0.0440, -0.1170,  0.1160, -0.1050, -0.0160, -0.0420, -0.0210, -0.0970]],
       grad_fn=<SliceBackward0>)

In [194]:
# def dReLU(X):
#     z = torch.zeros_like(X)
#     return X if (X >=0) else z
# dReLU(Z[2])
((Z[2]>=0) *A[2])

tensor([[0.0000, 0.0000, 0.0000, 0.2169, 0.0000, 0.0000, 0.0000, 0.0000, 0.0953,
         0.1247, 0.0766, 0.1419, 0.0661, 0.4032, 0.0039, 0.0347, 0.2394, 0.0705,
         0.0000, 0.1461, 0.0000, 0.0986, 0.0000, 0.0000, 0.0000, 0.0000, 0.0464,
         0.1349, 0.0000, 0.0000, 0.2248, 0.1526, 0.0000, 0.0000, 0.3292, 0.1120,
         0.0000, 0.0000, 0.0845, 0.0723],
        [0.0000, 0.0000, 0.0000, 0.1868, 0.0000, 0.0000, 0.0000, 0.0000, 0.1488,
         0.2620, 0.0000, 0.1329, 0.0912, 0.4439, 0.0000, 0.0930, 0.0983, 0.0000,
         0.0158, 0.3169, 0.0845, 0.0402, 0.0000, 0.0000, 0.0000, 0.0000, 0.0538,
         0.2010, 0.0000, 0.0758, 0.2067, 0.0000, 0.0000, 0.0000, 0.3267, 0.0000,
         0.0000, 0.0000, 0.1636, 0.0000],
        [0.1823, 0.0000, 0.0000, 0.0882, 0.0273, 0.0000, 0.0000, 0.0000, 0.0276,
         0.1786, 0.0000, 0.1130, 0.0395, 0.2506, 0.0000, 0.0000, 0.0719, 0.0000,
         0.0932, 0.0816, 0.0000, 0.1422, 0.1062, 0.0000, 0.0000, 0.0000, 0.0000,
         0.1407, 0.0000, 